In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Input, Dense, Reshape, Conv2DTranspose,\
   Activation, BatchNormalization, ReLU, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
import cv2
%matplotlib inline
import os

In [8]:
devices=tf.config.experimental.list_physical_devices('GPU')
print("GPUS: ", len(devices))
tf.config.experimental.set_memory_growth(devices[0],True)

GPUS:  1


In [9]:
def load_data():
    trn_x_names=os.listdir('data/train_x')
    trn_y_names=os.listdir('data/train_y')
    
    trn_x_list=[]
    trn_y_list=[]
    
    for image in trn_x_names[0:]:
        img=cv2.imread('data/train_x/%s'%image)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        trn_x_list.append(img)
        
    for image in trn_y_names[0:]:
        img=cv2.imread('data/train_y/%s'%image)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        trn_y_list.append(img)
        
    x_train= np.array(trn_x_list)
    y_train= np.array(trn_y_list)
    return x_train,y_train


def plotImages(images_arr):
  fig,axes = plt.subplots(1,10,figsize(20,20))
  axes=axes.flatten()
  for img,ax in zip(images_arr,axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
x_train,y_train=load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.
y_train = y_train.astype('float32') / 255.

In [ ]:
def conv_operation(x, filters, kernel_size, strides=1):
   x = Conv2D(filters=filters,
              kernel_size=kernel_size,
              strides=strides,
              padding='same')(x)
   x = BatchNormalization()(x)
   x = ReLU()(x)
   return x

def conv_transpose_operation(x, filters, kernel_size,strides=1):
   x = Conv2DTranspose(filters=filters,
                       kernel_size=kernel_size,
                       strides=strides,
                       padding='same')(x)
   x = BatchNormalization()(x)
   x = ReLU()(x)
   return x